In [1]:
import random
#For dealing with tables
import pandas as pd
#For dealing with linear algebra
import numpy as np
#For data visualization and plotting graphs
import matplotlib.pyplot as plt
import seaborn as sns
#For minmaxscaler
from sklearn.preprocessing import MinMaxScaler
#For shapiro test
from scipy.stats import shapiro
#For train-test split
from sklearn.model_selection import train_test_split,GridSearchCV
#For RandomForest
from sklearn.ensemble import RandomForestClassifier
#Accuracy score, confusion matrix, classification report, ROC curve, AUC
from sklearn.metrics import f1_score
#To ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Train Data
train_df = pd.read_csv("/content/train_MpHjUjU.csv")
#Test Data
test_df = pd.read_csv("/content/train_MpHjUjU.csv")

In [3]:
#Copy of train_df
train_data=train_df.copy()
#Copy of test_df
test_data=test_df.copy()

In [4]:
#The first 5 rows of train_data
train_data.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,01-01-2016,1,28,Male,C23,Master,57387,24-12-2015,NaN,1,1,2381060,2
1,01-02-2016,1,28,Male,C23,Master,57387,24-12-2015,NaN,1,1,-665480,2
2,01-03-2016,1,28,Male,C23,Master,57387,24-12-2015,11-03-2016,1,1,0,2
3,01-11-2017,2,31,Male,C7,Master,67016,06-11-2017,NaN,2,2,0,1
4,01-12-2017,2,31,Male,C7,Master,67016,06-11-2017,NaN,2,2,0,1


In [5]:
#Features of the train data
print("The features or column names of the train data are : \n",train_data.columns)

The features or column names of the train data are : 
 Index(['MMM-YY', 'Emp_ID', 'Age', 'Gender', 'City', 'Education_Level',
       'Salary', 'Dateofjoining', 'LastWorkingDate', 'Joining Designation',
       'Designation', 'Total Business Value', 'Quarterly Rating'],
      dtype='object')


In [6]:
##Converting 'MMM-YY' feature to datetime type
train_data['MMM-YY'] = pd.to_datetime(train_data['MMM-YY'],format="%d-%m-%Y")

##Converting 'Dateofjoining' feature to datetime type
train_data['Dateofjoining'] = pd.to_datetime(train_data['Dateofjoining'],format="%d-%m-%Y")

##Converting 'LastWorkingDate' feature to datetime type
train_data['LastWorkingDate'] = pd.to_datetime(train_data['LastWorkingDate'],format="%d-%m-%Y")

In [7]:
#Checking if the changes are done
train_data.dtypes

MMM-YY                  datetime64[ns]
Emp_ID                           int64
Age                              int64
Gender                          object
City                            object
Education_Level                 object
Salary                           int64
Dateofjoining           datetime64[ns]
LastWorkingDate         datetime64[ns]
Joining Designation              int64
Designation                      int64
Total Business Value             int64
Quarterly Rating                 int64
dtype: object

In [8]:
#Shape of the train data
print("train_data shape : ",train_data.shape)

#Shape of the test data
print("test_data shape : ",test_data.shape)

train_data shape :  (19104, 13)
test_data shape :  (19104, 13)


In [9]:
#Creating an empty dataframe
df = pd.DataFrame()

In [10]:
#Setting the emp_id as the new column which are unique
df['Emp_ID'] = train_data['Emp_ID'].unique()

#Setting age of the employee which takes the maximum age of the employee
df['Age'] = list(train_data.groupby('Emp_ID',axis=0).max('MMM-YY')['Age'])

#Setting gender of the employee 
df['Gender'] = list(train_data.groupby('Emp_ID').agg({'Gender':'first'})['Gender'])

#Setting the city of the employee 
df['City'] = list(train_data.groupby('Emp_ID').agg({'City':'first'})['City'])

#Setting the education of the employee 
df['Education'] = list(train_data.groupby('Emp_ID').agg({'Education_Level':'last'})['Education_Level'])

#Setting the salary of the employee for one month
df['Salary'] = list(train_data.groupby('Emp_ID').agg({'Salary':'last'})['Salary'])

#Setting the joining designtion of the employee 
df['Joining_Designation'] = list(train_data.groupby('Emp_ID').agg({'Joining Designation':'last'})['Joining Designation'])

#Setting the designtion of the employee at the time of reporting
df['Designation'] = list(train_data.groupby('Emp_ID').agg({'Designation':'last'})['Designation'])

#Setting the total business value acquired by the employee
df['Total_Business_Value'] = list(train_data.groupby('Emp_ID',axis=0).sum('Total Business Value')['Total Business Value'])

##Setting the quarterly rating of the employee
df['Last_Quarterly_Rating'] = list(train_data.groupby('Emp_ID').agg({'Quarterly Rating':'last'})['Quarterly Rating'])

In [11]:
#Creating a column which tells if the quarterly rating has increased for that employee 
#for those whose quarterly rating has increased we assign the value 1

#Quarterly rating at the beginning
qrf = train_data.groupby('Emp_ID').agg({'Quarterly Rating':'first'})

#Quarterly rating at the end
qrl = train_data.groupby('Emp_ID').agg({'Quarterly Rating':'last'})

#The dataset which has the employee ids and a bollean value which tells if the rating has increased
qr = (qrl['Quarterly Rating']>qrf['Quarterly Rating']).reset_index()

#the employee ids whose rating has increased
empid = qr[qr['Quarterly Rating']==True]['Emp_ID']

qri = []
for i in df['Emp_ID']:
    if i in empid:
        qri.append(1)
    else:
        qri.append(0)

df['Quarterly_Rating_Increased'] = qri

In [12]:
#Creating a column called target which tells if the person has left the company
#persons who have a last working date will have the value 1

#The dataset which has the employee ids and specifies if last working date is null
lwr = (train_data.groupby('Emp_ID').agg({'LastWorkingDate':'last'})['LastWorkingDate'].isna()).reset_index()

#The employee ids who do not have last working date
empid = list(lwr[lwr['LastWorkingDate']==True]['Emp_ID'])

target = []
for i in df['Emp_ID']:
    if i in empid:
        target.append(0)
    elif i not in empid:
        target.append(1)
        
df['Target'] = target

In [18]:
#Creating a column which tells if the monthly income has increased for that employee 
#for those whose monthly income has increased we assign the value 1

#Quarterly rating at the beginning
sf = train_data.groupby('Emp_ID').agg({'Salary':'first'})

#Quarterly rating at the end
sl = train_data.groupby('Emp_ID').agg({'Salary':'last'})

#The dataset which has the employee ids and a bollean value which tells if the monthly income has increased
s = (sl['Salary']>sf['Salary']).reset_index()

#the employee ids whose monthly income has increased
empid = s[s['Salary']==True]['Emp_ID']

si = []
for i in df['Emp_ID']:
    if i in empid:
        si.append(1)
    else:
        si.append(0)

df['Salary_Increased'] = si

In [19]:
df.head()

,Emp_ID,Age,Gender,City,Education,Salary,Joining_Designation,Designation,Total_Business_Value,Last_Quarterly_Rating,Quarterly_Rating_Increased,Target,Salary_Increased
0,1,28,Male,C23,Master,57387,1,1,1715580,2,0,1,0
1,2,31,Male,C7,Master,67016,2,2,0,1,0,0,0
2,4,43,Male,C13,Master,65603,2,2,350000,1,1,1,0
3,5,29,Male,C9,College,46368,1,1,120360,1,0,1,0
4,6,31,Female,C11,Bachelor,78728,3,3,1265000,2,0,0,0


In [20]:
df.describe()

,Emp_ID,Age,Salary,Joining_Designation,Designation,Total_Business_Value,Last_Quarterly_Rating,Quarterly_Rating_Increased,Target,Salary_Increased
count,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2.381000e+03,2381.000000,2381.000000,2381.000000,2381.000000
mean,1397.559009,33.663167,59334.157077,1.820244,2.096598,4.586742e+06,1.427971,0.128097,0.678706,0.015120
std,806.161628,5.983375,28383.666384,0.841433,0.941522,9.127115e+06,0.809839,0.334268,0.467071,0.122055
min,1.000000,21.000000,10747.000000,1.000000,1.000000,-1.385530e+06,1.000000,0.000000,0.000000,0.000000
25%,695.000000,29.000000,39104.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000
50%,1400.000000,33.000000,55315.000000,2.000000,2.000000,8.176800e+05,1.000000,0.000000,1.000000,0.000000
75%,2100.000000,37.000000,75986.000000,2.000000,3.000000,4.173650e+06,2.000000,0.000000,1.000000,0.000000
max,2788.000000,58.000000,188418.000000,5.000000,5.000000,9.533106e+07,4.000000,1.000000,1.000000,1.000000


In [21]:
df.describe(include=['O'])

,Gender,City,Education
count,2381,2381,2381
unique,2,29,3
top,Male,C20,Master
freq,1404,152,802


In [22]:
df['Target'].value_counts()

1    1616
0     765
Name: Target, dtype: int64

In [23]:
df['Target'].value_counts(normalize=True)

1    0.678706
0    0.321294
Name: Target, dtype: float64

In [28]:
#Gender encoding
df['Gender'] = df['Gender'].astype('category')
df['Gender_encode'] = df['Gender'].cat.codes

In [29]:
#One hot encoding for City, Education, Joining_Designation, Designation, Last_Quarterly_Rating
df = pd.concat([df,pd.get_dummies(df['City'],prefix='City')],axis=1)

df = pd.concat([df,pd.get_dummies(df['Education'],prefix='Education')],axis=1)

df = pd.concat([df,pd.get_dummies(df['Joining_Designation'],prefix='JDesignation')],axis=1)

df = pd.concat([df,pd.get_dummies(df['Designation'],prefix='Designation')],axis=1)

df = pd.concat([df,pd.get_dummies(df['Last_Quarterly_Rating'],prefix='LQR')],axis=1)

In [30]:
#MinMaxScaler
scaler = MinMaxScaler()

#Mathematically learning the distribution
df['Age_scaled'] = scaler.fit_transform((df['Age'].values).reshape(-1,1))
df['Salary_scaled'] = scaler.fit_transform((df['Salary'].values).reshape(-1,1))
df['Business _Value_scaled']=scaler.fit_transform((df['Total_Business_Value'].values).reshape(-1,1))

In [31]:
df1=df.copy()
df.drop(['Emp_ID', 'Age', 'Gender', 'City', 'Education', 'Salary',
       'Joining_Designation', 'Designation', 'Total_Business_Value'],axis=1,inplace=True)

In [32]:
#Feature Variables
X = df.drop('Target',axis=1)

#Target Variable
y = df['Target']

Modelling

In [33]:
param = {'max_depth':[4,5,6,7,8,9,10], 'n_estimators':[50,100,150,200,250]}

random_forest = RandomForestClassifier()

c = GridSearchCV(random_forest,param,cv=5,scoring='f1')
c.fit(X,y)

def display(results):
    print(f'Best parameters are : {results.best_params_}')
    print(f'The score is : {results.best_score_}')
display(c)

Best parameters are : {'max_depth': 8, 'n_estimators': 250}
The score is : 0.8696675903768589


In [34]:
decision_tree = RandomForestClassifier(max_depth=10, n_estimators=250)
rf_model=decision_tree.fit(X,y)

In [37]:
pred = rf_model.predict(X)
print("The f1-score = ",f1_score(y,pred))
df1['Pred'] = pred

The f1-score =  0.8991981672394044


In [41]:
submission = test_df.copy()
empid = submission['Emp_ID']

tr = []

for i in empid:
    tr.append(df1[df1['Emp_ID']==i]['Pred'].item())

In [42]:
df1

,Emp_ID,Age,Gender,City,Education,Salary,Joining_Designation,Designation,Total_Business_Value,Last_Quarterly_Rating,Quarterly_Rating_Increased,Target,Salary_Increased,Gender_encode,City_C1,City_C10,City_C11,City_C12,City_C13,City_C14,City_C15,City_C16,City_C17,City_C18,City_C19,City_C2,City_C20,City_C21,City_C22,City_C23,City_C24,City_C25,City_C26,City_C27,City_C28,City_C29,City_C3,City_C4,City_C5,City_C6,City_C7,City_C8,City_C9,Education_Bachelor,Education_College,Education_Master,JDesignation_1,JDesignation_2,JDesignation_3,JDesignation_4,JDesignation_5,Designation_1,Designation_2,Designation_3,Designation_4,Designation_5,LQR_1,LQR_2,LQR_3,LQR_4,Age_scaled,Salary_scaled,Business _Value_scaled,Pred
0,1,28,Male,C23,Master,57387,1,1,1715580,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0.189189,0.262508,0.032064,1
1,2,31,Male,C7,Master,67016,2,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0.270270,0.316703,0.014326,1
2,4,43,Male,C13,Master,65603,2,2,350000,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0.594595,0.308750,0.017944,1
3,5,29,Male,C9,College,46368,1,1,120360,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0.216216,0.200489,0.015570,1
4,6,31,Female,C11,Bachelor,78728,3,3,1265000,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0.270270,0.382623,0.027405,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2376,2784,34,Male,C24,College,82815,2,3,21748820,4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0.351351,0.405626,0.239197,0
2377,2785,34,Female,C9,College,12105,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0.351351,0.007643,0.014326,1
2378,2786,45,Male,C19,College,35370,2,2,2815090,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0.648649,0.138588,0.043432,1
2379,2787,28,Female,C20,Master,69498,1,1,977830,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0.189189,0.330673,0.024436,1


In [40]:
submission.shape

(741, 1)